MNIST Direct
============

This walk-through demonstrates the flexiblity of ngraph and how users can build the MNIST multi-layer perceptron model using ngraph components only. **For a simpler example, see the `mnist_mlp.py` example that uses the neon frontend to build the identical model.**

Note: here we do use neon components for downloading the MNIST data and providing minibatches of data to the ngraph model using neon's `ArrayIterator` object. However, the model definition and training is done with the ngraph API directly.

MNIST is a computer vision dataset consisting of 70,000 images of handwritten digits. Each image has 28x28 pixels for a total of 784 features, and is associated with a digit between 0-9.

<img src="http://corpocrat.com/wp-content/uploads/2014/10/figure_1.png" width=200px>

Setup data and axes
-------------------

We first use neon components to load the data.

In [ ]:
from __future__ import division
from __future__ import print_function
import numpy as np
from ngraph.frontends.neon import ArrayIterator, NgraphArgparser
from mnist import MNIST

np.random.seed(0)
batch_size = 128

# Create the dataloader
train_data, valid_data = MNIST('~/nervana/data/').load_data()

This function automatically splits the images `X` and labels `y` into training `(60,000 examples)` and testing `(10,000 examples)` data. The training images `train_data` is a numpy array with shape `(num_examples, num_features) = (60000, 1, 28, 28)`.
During training, neon iterates over the training examples to compute the gradients. We use the following commands to set up the `ArrayIterator` object that handles sending data to the model.

In [ ]:
train_set = ArrayIterator(train_data, batch_size, total_iterations=2000)
valid_set = ArrayIterator(valid_data, batch_size)

num_batches = np.floor(train_set.ndata/batch_size)

Next, we create the axes needed by the model. As before, we use the `make_name_scope` for convenience. There are several important axes here:
- (`C`, `H`, `W`, `N`): the shape of the input data in (channels, height, width, batch_size)
- (`M`): the number of hidden units in our affine layer.
- (`Y`): the shape of the output of the model, which is equal to the number of classes

In [ ]:
import ngraph as ng
import ngraph.transformers as ngt

ax = ng.make_name_scope(name="ax")
ax.N = ng.make_axis(batch=True, docstring="minibatch size")
ax.C = ng.make_axis(docstring="channels")
ax.H = ng.make_axis(docstring="image height")
ax.W = ng.make_axis(docstring="image width")
ax.Y = ng.make_axis(docstring="target shape")

ax.M = ng.make_axis(100, docstring="hidden layer nodes")

From our input data, we can already set the length of many of these axes. Because `M` is user defined, 

In [ ]:
ax.C.length, ax.H.length, ax.W.length = train_set.shapes['image']
ax.N.length = batch_size
ax.Y.length = 10

We can then create the needed placeholders and variables for defining the model:
- `x`: placeholder for the input image
- `t`: placeholder for the input target
- `w1`: weight matrix between the input image and the hidden layer.
- `w2`: weight matrix between the hidden layer and the output layer.

Defining the placeholders is relatively straightforward. Note that the ordering for `x` matters since the data from `ArrayIterator` is provided in the `CHWN` format.

In [ ]:
x = ng.placeholder([ax.C, ax.H, ax.W, ax.N])
t = ng.placeholder([ax.N])

We define the weights as `variables` and initialize them as random gaussian variables.

In [ ]:
w1_axes = ng.make_axes([ax.M, ax.C - 1, ax.H - 1, ax.W - 1])
w1 = ng.variable(initial_value=np.random.normal(0.0, 0.01, w1_axes.lengths), axes=w1_axes)

w2_axes = ng.make_axes([ax.Y, ax.M - 1])
w2 = ng.variable(initial_value=np.random.normal(0.0, 0.01, w2_axes.lengths), axes=w2_axes)

The nervana graph axes are agnostic to data layout on the compute device, so the ordering of the axes does not matter. As a consequence, when two tensors are provided to a `ng.dot()` operation, for example, one needs to indicate which are the corresponding axes that should be matched together. We use "dual offsets" of +/- 1 to mark which axes should be matched during a multi-axis operation, which gives rise to the `ax.C - 1` observed above.

For more information on dual axes, see the `Axes` section of the docs.

Network
-------

We then define our network using the ngraph operations to obtain the activations from the hidden layer `h1` and the output layer `h2`.

The hidden layer is expressed as a Linear layer followed by a rectified linear activation function. The output layer is also linear, but with a sigmoid activation function.

In [ ]:
h1 = ng.maximum(ng.dot(w1, x / 255.), 0)
h2 = ng.sigmoid(ng.dot(w2, h1))

Cost
----

To train the network, we use cross entropy binary loss function. We also define some useful computations for obtaining the total loss, the mean loss, predictions, and errors.

In [ ]:
loss = ng.cross_entropy_binary(h2, ng.one_hot(t, axis=ax.Y))

# useful model operations
total_loss = ng.sum(loss, out_axes=())
mean_cost = ng.mean(loss, out_axes=())
predictions = ng.argmax(h2)
errors = ng.not_equal(predictions, t)

Optimizer
---------

Here we implement gradient descent with momentum using the ngraph API. Just as with the Logistic Regression example, we use `ng.deriv` to obtain the gradients for all the variables that we with to optimize over.

In [ ]:
variables = list(total_loss.variables())
grads = [ng.deriv(total_loss, variable) / ax.N.length for variable in variables]

For the SGD with momentum optimizer, we implement below the required steps to:
1. define the velocities for each variable as `persistent_tensor`.
2. update the velocities with each call according to $v' = v m - \alpha \nabla$
3. update the parameters via $W' = W + v$

In [ ]:
learning_rate = 0.1
momentum = 0.9

with ng.Op.saved_user_deps():
    velocities = [ng.persistent_tensor(axes=variable.axes, initial_value=0.)
                  for variable in variables]
    velocity_updates = [ng.assign(velocity,
                                  velocity * momentum - learning_rate * grad)
                        for velocity, grad in zip(velocities, grads)]
    param_updates = [ng.assign(variable, variable + velocity)
                     for variable, velocity in zip(variables, velocities)]

updates = ng.doall(velocity_updates + param_updates)

Several of the ops above are defined with the `ng.Op.saved_user_deps()` context. Ops defined in this context are excluded as depend

We define two computations below, one for updating the weights and returning the average cost for each minibatch of data. The second computation obtains the misclassification error.

In [ ]:
transformer = ngt.make_transformer()
train_comp = transformer.computation([mean_cost, updates], x, t)
error_comp = transformer.computation(errors, x, t)
transformer.initialize()

Training
--------

Finally, we can train the model by iterating over the training set, calling `train_comp` for each minibatch of data. We also print the total cost at regular intervals.

In [ ]:
total_cost = []
for mb_idx, npbufs in enumerate(train_set):
    batch_cost, _ = train_comp(npbufs['image'], npbufs['label'])
    total_cost.append(float(batch_cost))

    if len(total_cost) == num_batches:
        print("[Epoch %s] Cost = %s" % ((mb_idx + 1) // num_batches, np.mean(total_cost)))
        total_cost = []

Validation
----------

Last, we take our trained graph, and evaluate the misclassification error on a held-out `valid_set`.

In [ ]:
running_error = 0.
all_errors = []
valid_set.reset()


for npbufs in valid_set:
    error_val = error_comp(npbufs['image'], npbufs['label'])
    all_errors.append(list(error_val))

all_errors = all_errors[:valid_set.ndata]  # Truncate to remove any leftovers

print('Validation Error = %s%%' % (np.mean(all_errors) * 100.,))